In [1]:
from pyfuseki import FusekiUpdate, FusekiQuery
import httpx
from rdflib import Graph
from pysolr import Solr
from api.src.schemas.thesaurus.subject import Subject_Edit

In [ ]:
obj = {
    "note": {
        "value": "Aqui são inseridos trabalhos na parte da World Wide Web que é intencionalmente oculta dos mecanismos de pesquisa e é acessível apenas com software de anonimato e autorizações especiais. Os trabalhos na parte da World Wide Web que podem não estar acessíveis por meio de mecanismos de pesquisa de uso geral são inseridos em [Web invisível.]",
        "lang": "pt"
    },
    "authority": {
        "value": "Dark Web",
        "lang": "pt"
    },
    "narrower": [],
    "broader": [
        {
            "value": "Web Invisível",
            "lang": "pt",
            "uri": "http://id.loc.gov/authorities/subjects/sh2008009697"
        }
    ],
    "variant": [
        {
            "value": "Internet escura",
            "lang": "pt",
            "type": "Topic"
        },
        {
            "value": "Darknet",
            "lang": "pt",
            "type": "Topic"
        }
    ],
    "reciprocalAuthority": [],
    "exactExternalAuthority": [
        "http://id.loc.gov/authorities/subjects/sh2018002121"
    ],
    "closeExternalAuthority": [
        "http://id.worldcat.org/fast/2003430"
    ],
    "tokenLSCH": "sh2018002121"
}

In [2]:
g = Graph()
g.parse("http://id.loc.gov/authorities/subjects/sh2018002121.rdf")
g.serialize('dark_web.nt')

<Graph identifier=N0200e03371de439eb99d76a0224276fc (<class 'rdflib.graph.Graph'>)>

In [56]:
uri = f'http://localhost:3030/authorities?graph=https:%2F%2Fbibliokeia.com%2Fauthorities%2Fsubjects%2Fsh2018002121'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('subjects.nt')

<Graph identifier=N6ed80c88751e455889d8bba1976f7f33 (<class 'rdflib.graph.Graph'>)>

In [6]:
upAuthorities = FusekiUpdate('http://localhost:3030', 'authorities')

In [3]:
G1 = """PREFIX : <http://example/>
            PREFIX subjects: <https://bibliokeia.com/authorities/subjects/>

    INSERT DATA {
        GRAPH subjects:TESTE { :x :p (1 2 3) . } }"""

response = upAuthorities.run_sparql(G1)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [4]:
uri = f'http://localhost:3030/authorities?graph=https://bibliokeia.com/authorities/subjects/TESTE'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('teste.nt')

<Graph identifier=N4c274385fd8c44bda05253763df99054 (<class 'rdflib.graph.Graph'>)>

In [7]:
up = """PREFIX :    <http://example/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX subjects: <https://bibliokeia.com/authorities/subjects/>

WITH subjects:TESTE

DELETE { ?x :p ?list }
INSERT { ?x :p [ rdf:first 'ZERO' ; 
                 rdf:rest ?list ]
} WHERE {
  ?x :p ?list .
}"""

response = upAuthorities.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [8]:
up = """PREFIX :    <http://example/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX subjects: <https://bibliokeia.com/authorities/subjects/>

WITH subjects:TESTE

DELETE { 
   ?x :p ?list .
   ?list rdf:first ?first ;
         rdf:rest  ?rest }
INSERT { ?x :p ?rest }
WHERE
{
  ?x :p ?list .
  ?list rdf:first ?first ;
        rdf:rest ?rest .
}"""

response = upAuthorities.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [5]:
prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX work: <https://bibliokeia.com/resources/work/>
PREFIX subjects: <https://bibliokeia.com/authorities/subjects/>
PREFIX names: <https://bibliokeia.com/authorities/names/>
PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>"""

In [41]:
bkID = "sh2018002121"
up = prefix+"WITH subjects:"+bkID+"""

        DELETE { subjects:sh2018002121 madsrdf:hasVariant ?hasVariant .
    ?hasVariant ?p1 ?variant .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element . 
    ?elementList rdf:rest rdf:nil . 
    ?element ?p3 ?o
      }             
        WHERE { subjects:sh2018002121 madsrdf:hasVariant ?hasVariant .
    ?hasVariant ?p1 ?variant .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList rdf:first ?element . 
    ?elementList rdf:rest rdf:nil . 
    ?element ?p3 ?o
                        }"""
response = upAuthorities.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [54]:
insert = prefix+"""INSERT DATA
{ GRAPH subjects:"""+bkID+""" { 
    subjects:sh2018002121 madsrdf:hasVariant [
    rdf:type madsrdf:Topic;
    rdf:type madsrdf:Variant ;
    madsrdf:elementList ( [
    rdf:type madsrdf:TopicElement;
    madsrdf:elementValue 'NOVA VARIANTE'
    ]);
    madsrdf:variantLabel 'NOVA VARIANTE'
    ] 
 } }"""

response = upAuthorities.run_sparql(insert)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [ ]:
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX ns: <http://example.org/ns#>
INSERT DATA
{ GRAPH <http://example/bookStore> { <http://example/book1>  ns:price  42 } }

In [53]:
d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAuthorities.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [57]:
bkID = "sh2018002121"
up = prefix+"WITH subjects:"+bkID+"""

        DELETE { subjects:sh2018002121 madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList ?p ?element .
    ?element madsrdf:elementValue 'NOVA VARIANTE'
      }   
      INSERT {
      subjects:sh2018002121 madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList ?p ?element .
    ?element madsrdf:elementValue 'VARIANTE'      
      }         
        WHERE { subjects:sh2018002121 madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:elementList ?elementList .
    ?elementList ?p ?element .
    ?element madsrdf:elementValue 'NOVA VARIANTE'
                        }"""
response = upAuthorities.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [58]:
bkID = "sh2018002121"
up = prefix+"WITH subjects:"+bkID+"""

        DELETE { subjects:sh2018002121 madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:variantLabel "NOVA VARIANTE" .
      }   
      INSERT {
      subjects:sh2018002121 madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:variantLabel "VARIANTE" .      
      }         
        WHERE { subjects:sh2018002121 madsrdf:hasVariant ?hasVariant .
    ?hasVariant madsrdf:variantLabel "NOVA VARIANTE" .
                        }"""
response = upAuthorities.run_sparql(up)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}